## Assignments:  implement  Binomial , Poisson and co-relation function with Python 

###### Python function  to calculate Binomial probabilities and other statistical parameters 

In [1]:
def binomial(data,ret,poslabel,dic,prob=None):
    """
    input : 
        data     : list of data population or int representing Sample population
        ret      : int or string where 1: Ex(X), 2:Var(X),3:standard_devaiation, All: will fetch (1,2,3)
        poslabel : int or string label in data for identifies postive cases
        dic      : (key,value)pairs for computing cummulative probabilities of cases  possible keys [less,equal,greater] 
        prob     : prob of data is know else probability( pos) is caluclated from data if passed
    out put : 
        dictionary :
         will have default key,value pair which indicates the probability of cases default is 0
         
    Example1: binomial([1,2,3,4,5,1,1,1,1,1],ret="all",poslabel=1,dic={"equal":2})
             result = {'E(1)': 6.0,'Var(1)': 2.4000000000000004,'StdDev(1)': 1.5491933384829668,
             'Probability of condition': 0.010616832000000005}
    Example2: binomial(8,"all","X",{"less":2},0.6)  
             result = {'E(X)': 4.8,
                        'Var(X)': 1.92,
                        'StdDev(X)': 1.3856406460551018,
                         'Probability of condition': 0.008519680000000003}
                         
    """
    import math
    output,data,n,pos,cumm_format,cmf = {},data,0,0,{},0
    binom_loop = {}
    
    if isinstance(data,int):
        n=data
    elif isinstance(data,list):
        n = len(data)
        
    if prob==None:
        pos = data.count(poslabel)/len(data)
    else:
        pos = prob
    
    ex_x,var_x,std_x=0,0,0
    ex_x =  n * pos
    var_x = n * pos * (1-pos)
    std_x = math.sqrt(var_x)
    
    def cummulative(data,probability,condition):
        loop,result = 0,0
        
        def fact(x):
            fac =1
            for i in range(x,0,-1):
                fac= fac * i 
            return fac
    
        probs = {}
        for i in range(data,-1,-1):
            n = data
            x = i
            probs[i]=fact(n)/(fact(i)*fact(n-i)) * (probability) ** i * (1-probability) ** (n-i)
        cond = list(condition.keys())[0]
        if cond =="greater":
            for i in range(condition[cond]+1,data+1):
                result=result+probs[i]
        elif cond == "less":
            for i in range(condition[cond]-1,-1,-1):
                result = result+probs[i]
        elif cond == "equal":
            result = result+probs[condition[cond]]
        return result
    
    
    if len(dic)==0:
        cumm_format = {"default":cmf}
    else:
        cmf = cummulative(n,pos,dic)
        cumm_format = {"{}".format(list(dic.keys())[0]):cmf}
    
    if isinstance(ret,str):
        output= {"E({})".format(poslabel):ex_x,"Var({})".format(poslabel):var_x,
                "StdDev({})".format(poslabel):std_x,"Probability of condition":cmf}
    elif ret == 1:
        output = {"E({})".format(poslabel):ex_x,"Probability of condition":cmf}
    
    elif ret == 2:
        output = {"Var({})".format(poslabel):var_x,"Probability of condition":cmf}
    
    elif ret == 3:
        output = {"StdDev({})".format(poslabel):std_x,"Probability of condition":cmf}
    
    
    return output       

In [2]:
binomial(8,"all","X",{"less":2},0.6)

{'E(X)': 4.8,
 'Var(X)': 1.92,
 'StdDev(X)': 1.3856406460551018,
 'Probability of condition': 0.008519680000000003}

In [3]:
binomial(8,1,"X",{"less":2},0.6)

{'E(X)': 4.8, 'Probability of condition': 0.008519680000000003}

###### Poisson distribution and  probabilites

In [4]:
## Poisson distribution

def poisson(rate,case):
    """
        inputs:
            rate: rate 
            case: list -> list of K or (key,values) for caluclating the different rates
        output:
            dictionary:
        example:poisson(10,[0,1,2])
                output:{'P(0)': 4.5399929762484875e-05,
                        'P(1)': 0.00045399929762484877,
                        'P(2)': 0.0022699964881242435}
                        
        example: poisson(2,{4:3,6:3}) 
                output:{'P(X =4 | 3)': 0.1338526175399834, 'P(X =6 | 3)': 0.1606231410479801}
    
    """
    
    import math
    #mean = rate * N
    #std = math.sqrt(mean)
    output = {}
    def fact(x):
        fac =1
        for i in range(x,0,-1):
            fac= fac * i 
        return fac
    #formula for poission : lamda ** case * e ** -lamda / k!
    if isinstance(case,list):
        for i in range(len(case)):
            output["P({})".format(case[i])] = rate ** case[i] * math.e ** -rate / fact(case[i])
    elif isinstance(case,dict):
        for i in range(len(case.keys())):
            key = list(case.keys())[i]
            val = case[key]
            lam = rate * val
            output["P(X ={}|{})".format(key,val)] = (lam )** key * math.e **-(lam) / fact(key)
            
    return output
    

In [5]:
poisson(2,{4:3,6:3})

{'P(X =4|3)': 0.1338526175399834, 'P(X =6|3)': 0.1606231410479801}

In [6]:
poisson(10,[0,1,2])

{'P(0)': 4.5399929762484875e-05,
 'P(1)': 0.00045399929762484877,
 'P(2)': 0.0022699964881242435}

###### Pearsons correaltion factor r 

In [7]:
## Correlation 

def correlation(data):
    """
        input : dictionary keys: values , values 1D list
        ouput : dictionary of pearsons values between data columns
    
    """
    import math
    m = len(data)
    n = len(data[list(data.keys())[0]])
    cols = list(data.keys())
    output ={"C":cols}
    
    for i in range(m):
        output[cols[i]]= []
        for j in range(m):
            x = data[cols[i]]
            y = data[cols[j]]
            x_sum = sum(x)
            y_sum = sum(y)
            xy_sum = sum([x[z] * y[z] for z in range(len(x))])
            sqrd_x = sum([z **2 for z in x])
            sqrd_y = sum([z ** 2 for z in y ])
            #pemdas
            numerator  = xy_sum - x_sum * y_sum / n
            denom_1 = sqrd_x - (x_sum ** 2)/ n
            denom_2 = sqrd_y - (y_sum ** 2) / n
            
            r =round(numerator / math.sqrt(denom_1 * denom_2),2)
            output[cols[i]].append(r)
            
            
    return output
    

In [8]:
data  = {"x":[7,6,8,5,6,9],"y":[12,8,12,10,11,13],"z":[7,6,8,5,6,9]}
data2 = {"x":[10,8,2,1,5,6],"y":[2,3,9,7,6,5]}

In [33]:
cor = correlation(data2)
for k,v in cor.items():
    print(k,v)

C ['x', 'y']
x [1.0, -0.94]
y [-0.94, 1.0]


In [9]:
corr = correlation(data)
for k,v in corr.items():
    print(k,v)

C ['x', 'y', 'z']
x [1.0, 0.76, 1.0]
y [0.76, 1.0, 0.76]
z [1.0, 0.76, 1.0]
